In [ ]:
%pylab inline
from parcels import FieldSet, Field, ParticleSet, JITParticle, AdvectionRK4, ErrorCode, Variable
#import cartopy
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import xarray as xr
from ParticlePlotFunctions import *
%matplotlib inline

In [ ]:
#load data
namefile = '../../results/data_output/galapagosparticles_testrun.nc'
Traj = ReadTrajectories(namefile)

# old

In [ ]:
find which particles arrived at the Galapagos islands
Traj['traveltime'] = [] #age at arrival
Traj['visittime'] = []  #date of arrival
Traj['pvisited'] = []   #index particle
Traj['tvisited'] = []   #index time
for p in range(Traj['visitedgalapagos'].shape[0]):               # loop through particles
    if np.any(Traj['visitedgalapagos'][p, :] == 1):              # check whether visited Galapagos
        I = np.where(Traj['visitedgalapagos'][p, :] == 1)[0][0]  # find first index when
        Traj['traveltime'].append(Traj['age'][p, I]/86400.) 
        Traj['visittime'].append(Traj['time'][p,I])
        Traj['pvisited'].append(p)       
        Traj['tvisited'].append(I)
                      

In [ ]:
#individual trajectories that reach Galapagos extent
plat = Traj['lat'][Traj['pvisited'],:]
plon = Traj['lon'][Traj['pvisited'],:]
map_extent = [-180,-70,-30,30]
PlotTrajectories(plon,plat,map_extent)
plt.savefig('../results/figures/TrajAmerica')

In [ ]:
#2D histogram from start to arrival

projection = cartopy.crs.PlateCarree(central_longitude=0)
fig, ax = plt.subplots(subplot_kw={'projection': projection}, figsize=(7,7))
    
grd = ax.gridlines(draw_labels=True,
                   color='gray', 
                   alpha=0.5, 
                   linestyle='--')
grd.xlabels_top = False
grd.ylabels_right = False
grd.xlabel_style = {'size': 15, 'color': 'black'}  
grd.ylabel_style = {'size': 15, 'color': 'black'}
ax.coastlines()
ax.add_feature(cartopy.feature.LAND, facecolor=(160/255, 160/255, 160/255))
ax.set_extent([-130,-60,-25,30]) 

bins = [np.arange(-130,-60, 1), np.arange(-25, 30, 1)]

temp = Traj['lon'][Traj['pvisited'],:]
plon = np.copy(temp)
temp = Traj['lat'][Traj['pvisited'],:]
plat = np.copy(temp)

for i, t in enumerate(Traj['tvisited']):
    plon[i,t:] = nan
    plat[i,t:] = nan

H, xe, ye = np.histogram2d(plon[~np.isnan(plon)], plat[~np.isnan(plat)], bins=bins)
H[0,: ] = H[-1, :]
xb = (xe[1:] + xe[:-1])/2
yb = (ye[1:] + ye[:-1])/2
levels = [10**x for x in range(0, 9)]

co = ax.contourf(xb, yb, H.T, levels=levels, norm=colors.LogNorm())


# Practice plotting
## Subplots


In [ ]:
%matplotlib inline

T = D['bwd_v2']
plat = T['lat']
plon = T['lon']
ptime = T['time']

projection = cartopy.crs.PlateCarree(central_longitude=0)

fig, ax = plt.subplots(1, 2, subplot_kw={'projection': projection}, figsize=(15,15))

for i in range (len(ax)):
    grd = ax[i].gridlines(draw_labels=True,
                          color='gray', 
                          alpha=0.5, 
                          linestyle='--')
    grd.xlabels_top = False
    grd.ylabels_right = False
    if i==1:
        grd.ylabels_left = False
    grd.xlabel_style = {'size': 15, 'color': 'black'}  
    grd.ylabel_style = {'size': 15, 'color': 'black'}
    ax[i].coastlines()
    ax[i].add_feature(cartopy.feature.LAND, facecolor=(160/255, 160/255, 160/255))
    ax[i].set_extent([-110, -70, -20, 30])    
    
particles = np.arange(1,300)
for particle in particles:
    ax[0].plot(plon[particle,:], plat[particle,:], 
               linewidth=1,
               color='r')

particles = np.arange(40000,40200)
for particle in particles:
    ax[1].plot(plon[particle,:],plat[particle,:])    
    
#plt.show()